In [1]:
#import sys
import numpy as np
import pandas as pd
%matplotlib inline
#import matplotlib.pyplot as plt
#import seaborn as sns
#import sklearn
import tensorflow as tf
import es_utils as utils

2022-03-25 08:55:45.488303: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-25 08:55:45.517841: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-25 08:55:45.517856: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-25 08:55:45.518246: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [2]:
'''labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)'''

labels = [
    'label:OR_standing',
    'label:SITTING',
    'label:LYING_DOWN',
    'label:FIX_running',
    'label:FIX_walking',
    'label:BICYCLING'
]


In [3]:
#k_folds = utils.create_k_folds_n_users(5, 40, '/home/wander/OtherProjects/har_flower/full_data')

In [4]:
config = {'df_path': '../input/user1.features_labels.csv'}
har = utils.HAR(config)
har.dummy_test()

Epoch 1/40
254/254 [==============================] - 2s 3ms/step - loss: 0.1553 - avg_multilabel_BA_2: 0.8704 - val_loss: 0.1190 - val_avg_multilabel_BA_2: 0.8963
Epoch 2/40
254/254 [==============================] - 0s 944us/step - loss: 0.1271 - avg_multilabel_BA_2: 0.8984 - val_loss: 0.1151 - val_avg_multilabel_BA_2: 0.8992
Epoch 3/40
254/254 [==============================] - 0s 918us/step - loss: 0.1060 - avg_multilabel_BA_2: 0.9024 - val_loss: 0.1339 - val_avg_multilabel_BA_2: 0.9041
Epoch 4/40
254/254 [==============================] - 0s 903us/step - loss: 0.1454 - avg_multilabel_BA_2: 0.9053 - val_loss: 0.1477 - val_avg_multilabel_BA_2: 0.9070
Epoch 5/40
254/254 [==============================] - 0s 913us/step - loss: 0.1353 - avg_multilabel_BA_2: 0.9083 - val_loss: 0.1467 - val_avg_multilabel_BA_2: 0.9099
Epoch 6/40
254/254 [==============================] - 0s 920us/step - loss: 0.1397 - avg_multilabel_BA_2: 0.9106 - val_loss: 0.1377 - val_avg_multilabel_BA_2: 0.9117
Epoch 

Epoch 9/40
199/199 [==============================] - 0s 896us/step - loss: 0.1901 - avg_multilabel_BA_2: 0.8963 - val_loss: 0.1833 - val_avg_multilabel_BA_2: 0.8953
Epoch 10/40
199/199 [==============================] - 0s 900us/step - loss: 0.1910 - avg_multilabel_BA_2: 0.8944 - val_loss: 0.1898 - val_avg_multilabel_BA_2: 0.8934
Epoch 11/40
199/199 [==============================] - 0s 901us/step - loss: 0.1954 - avg_multilabel_BA_2: 0.8925 - val_loss: 0.2042 - val_avg_multilabel_BA_2: 0.8915
Epoch 12/40
199/199 [==============================] - 0s 910us/step - loss: 0.1948 - avg_multilabel_BA_2: 0.8906 - val_loss: 0.1977 - val_avg_multilabel_BA_2: 0.8897
Epoch 13/40
199/199 [==============================] - 0s 928us/step - loss: 0.1908 - avg_multilabel_BA_2: 0.8888 - val_loss: 0.1901 - val_avg_multilabel_BA_2: 0.8880
Epoch 14/40
199/199 [==============================] - 0s 910us/step - loss: 0.1879 - avg_multilabel_BA_2: 0.8872 - val_loss: 0.1888 - val_avg_multilabel_BA_2: 0.8864

In [2]:
data = pd.read_csv('../input/user1.features_labels.csv')#.dropna() #TODO: attention

#data.isna().sum()
data.drop(columns=['lf_measurements:temperature_ambient'], inplace=True)
data[data.columns.drop(data.filter(regex='label:'))]

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,...,lf_measurements:screen_brightness,discrete:time_of_day:between0and6,discrete:time_of_day:between3and9,discrete:time_of_day:between6and12,discrete:time_of_day:between9and15,discrete:time_of_day:between12and18,discrete:time_of_day:between15and21,discrete:time_of_day:between18and24,discrete:time_of_day:between21and3,label_source
0,1464129912,1.011438,0.012573,0.023013,0.041240,1.006184,1.010868,1.016028,0.935099,6.684536,...,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
1,1464129950,1.011233,0.009356,-0.005622,0.016687,1.006338,1.010926,1.016657,1.732968,6.684569,...,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
2,1464130031,1.013422,0.018068,-0.008593,0.039286,1.004077,1.012983,1.021926,1.464639,6.684453,...,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
3,1464130109,1.014891,0.016400,0.021383,0.038825,1.005934,1.014670,1.023000,1.440043,6.684483,...,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
4,1464130130,1.017487,0.022632,-0.012891,0.037226,1.006040,1.017587,1.028168,1.937362,6.684364,...,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,1464886422,1.002114,0.043888,-0.023421,0.060480,0.975662,1.001421,1.028555,2.562570,6.683649,...,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
2681,1464886722,1.015957,0.068057,-0.030021,0.108892,0.980304,1.015067,1.050255,2.055226,6.682352,...,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
2682,1464887023,0.996011,0.039554,-0.034651,0.065742,0.976391,0.998637,1.018739,1.989673,6.683814,...,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
2683,1464887322,1.079283,0.404817,0.330815,0.569329,0.907679,1.001940,1.298900,2.453154,6.612619,...,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,4


In [4]:
#data[data.columns.drop(data.filter(regex='label:'))]
data.filter(regex='label:').columns.values.tolist()

['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'l

In [4]:
data[data.columns.drop(data.filter(regex='label:'))].dropna()

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,...,lf_measurements:screen_brightness,discrete:time_of_day:between0and6,discrete:time_of_day:between3and9,discrete:time_of_day:between6and12,discrete:time_of_day:between9and15,discrete:time_of_day:between12and18,discrete:time_of_day:between15and21,discrete:time_of_day:between18and24,discrete:time_of_day:between21and3,label_source


In [5]:
data.fillna(0.0, inplace=True)
#data.info()
data[data.columns.drop(data.filter(regex='label:'))].dropna()

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,...,lf_measurements:screen_brightness,discrete:time_of_day:between0and6,discrete:time_of_day:between3and9,discrete:time_of_day:between6and12,discrete:time_of_day:between9and15,discrete:time_of_day:between12and18,discrete:time_of_day:between15and21,discrete:time_of_day:between18and24,discrete:time_of_day:between21and3,label_source
0,1464129912,1.011438,0.012573,0.023013,0.041240,1.006184,1.010868,1.016028,0.935099,6.684536,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
1,1464129950,1.011233,0.009356,-0.005622,0.016687,1.006338,1.010926,1.016657,1.732968,6.684569,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
2,1464130031,1.013422,0.018068,-0.008593,0.039286,1.004077,1.012983,1.021926,1.464639,6.684453,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
3,1464130109,1.014891,0.016400,0.021383,0.038825,1.005934,1.014670,1.023000,1.440043,6.684483,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
4,1464130130,1.017487,0.022632,-0.012891,0.037226,1.006040,1.017587,1.028168,1.937362,6.684364,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,1464886422,1.002114,0.043888,-0.023421,0.060480,0.975662,1.001421,1.028555,2.562570,6.683649,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
2681,1464886722,1.015957,0.068057,-0.030021,0.108892,0.980304,1.015067,1.050255,2.055226,6.682352,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
2682,1464887023,0.996011,0.039554,-0.034651,0.065742,0.976391,0.998637,1.018739,1.989673,6.683814,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
2683,1464887322,1.079283,0.404817,0.330815,0.569329,0.907679,1.001940,1.298900,2.453154,6.612619,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,4


In [6]:
har.mlp.model.save('model/saved_model')

2022-03-11 17:23:04.276739: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model/saved_model/assets


In [9]:
converter = tf.lite.TFLiteConverter.from_saved_model('model/saved_model') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model/model.tflite', 'wb') as f:
  f.write(tflite_model)

2022-03-11 17:25:06.806404: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-03-11 17:25:06.806426: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-03-11 17:25:06.806430: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-03-11 17:25:06.807108: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/saved_model
2022-03-11 17:25:06.808538: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-03-11 17:25:06.808552: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: model/saved_model
2022-03-11 17:25:06.811993: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-03-11 17:25:06.837873: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: model/saved_model